Reference: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

# Imports

In [ ]:
import sys
from typing import List
import time

import torch
from torch import nn
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split, Subset

import numpy as np
import matplotlib.pyplot as plt
import heapq

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Change system path to base directory.
sys.path.append("..")
from preprocessing.preprocessor import Preprocessor
from datasets.fake_news_detection import FakeNewsDetectionDataset

# Manual set seed for reproducible results
torch.manual_seed(1)

## Read & process dataset

In [ ]:
# Control which linguistic preprocessing steps should run.
preprocessor = Preprocessor(perform_case_folding=False,
                            remove_stop_words=False,
                            remove_punctuation=False,
                            perform_lemmatization=False,
                            perform_stemming=False)

# Generate training and test datasets.
training_dataset = FakeNewsDetectionDataset("../raw_data/fulltrain.csv",
        transform=preprocessor.process, target_transform=lambda x: int(x) - 1)
test_dataset = FakeNewsDetectionDataset("../raw_data/balancedtest.csv",
        transform=preprocessor.process, target_transform=lambda x: int(x) - 1)

## Prepare data processing pipelines, generate data batch and iterator

In [ ]:
def yield_tokens(dataset: FakeNewsDetectionDataset) -> List[str]:
    for i in range(len(dataset)):
        sentence, _ = dataset[i]
        yield sentence

vocab = build_vocab_from_iterator(yield_tokens(training_dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
text_pipeline = lambda x: vocab(x)

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for text, label in batch:
        label_list.append(label)
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

## Training

### Model class

In [ ]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, 4)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

### Define train/eval functions, split training data

In [ ]:
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize).to(device)

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    tp, fp, fn = [0] * 4, [0] * 4, [0] * 4

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets).argmax(1)
            total_acc += (predicted_label == label).sum().item()
            total_count += label.size(0)

            for lab in range(4):
                tp[lab] += torch.mul(predicted_label == lab, label == lab).sum().item()
                fp[lab] += torch.mul(predicted_label == lab, label != lab).sum().item()
                fn[lab] += torch.mul(predicted_label != lab, label == lab).sum().item()

    f1s = [tp[lab] / (tp[lab] + 0.5 * (fp[lab] + fn[lab])) for lab in range(4)]
    return total_acc/total_count, f1s[0], f1s[1], f1s[2], f1s[3], sum(f1s)/4

# Hyperparameters
EPOCHS = 10 # epoch
LR = 0.001  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
num_train: int = int(len(training_dataset) * 0.95)
split_train_: Subset[FakeNewsDetectionDataset]
split_valid_: Subset[FakeNewsDetectionDataset]
split_train_, split_valid_ = random_split(training_dataset, [num_train, len(training_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

### Training Loop

In [ ]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val, f1_0, f1_1, f1_2, f1_3, f1_macro = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid accuracy {:8.3f} '
          '| f1 of class 0 {:8.3f} | f1 of class 1 {:8.3f} | f1 of class 2 {:8.3f} '
          '| f1 of class 3 {:8.3f} | macro f1 {:8.3f}'.format(epoch,
                                              time.time() - epoch_start_time,
                                              accu_val, f1_0, f1_1, f1_2, f1_3, f1_macro))
    print('-' * 59)

In [ ]:
# Save trained model
torch.save(model.state_dict(), '../trained_models/SNNModel-%sepoch.pt' % EPOCHS)

In [ ]:
# Load saved model to skip training
model.load_state_dict(torch.load('../trained_models/SNNModel-%sepoch.pt' % EPOCHS))

### Evaluating on test set

In [ ]:
print('Checking the results of test dataset.')
accu_test, f1_0_test, f1_1_test, f1_2_test, f1_3_test, f1_macro_test = evaluate(test_dataloader)
print('test accuracy {:8.3f} | test f1 of class 0 {:8.3f} | test f1 of class 1 {:8.3f} '
      '| test f1 of class 2 {:8.3f} | test f1 of class 3 {:8.3f} | test macro f1 {:8.3f}'.format(accu_test, f1_0_test, f1_1_test, f1_2_test, f1_3_test, f1_macro_test))

### Checking relationships between labels for documents

In [ ]:
CATEGORIES = { 1: 'Satire', 2: 'Hoax', 3: 'Propaganda', 4: 'Reliable news' }

reliable, hoax, satire, propaganda = [], [], [], []

all_documents_dataloader = DataLoader(training_dataset, batch_size=1,
                             shuffle=True, collate_fn=collate_batch)

with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(all_documents_dataloader):
        output = model(text, offsets)
        arr = output.numpy()[0]
        reliable.append(arr[3])
        hoax.append(arr[1])
        satire.append(arr[0])
        propaganda.append(arr[2])

In [ ]:
plt.scatter(reliable, hoax)
plt.xlabel('reliable')
plt.ylabel('hoax')
plt.title('Reliable vs hoax - documents')
plt.show()

plt.scatter(reliable, satire)
plt.xlabel('reliable')
plt.ylabel('satire')
plt.title('Reliable vs satire - documents')
plt.show()
plt.scatter(reliable, propaganda)
plt.title('Reliable vs propaganda - documents')
plt.xlabel('reliable')
plt.ylabel('propaganda')
plt.show()
plt.scatter(hoax, propaganda)
plt.title('Hoax vs propaganda - documents')
plt.xlabel('hoax')
plt.ylabel('propaganda')
plt.show()
plt.scatter(hoax, satire)
plt.title('Hoax vs satire - documents')
plt.xlabel('hoax')
plt.ylabel('satire')
plt.show()
plt.scatter(satire, propaganda)
plt.title('Satire vs propaganda - documents')
plt.xlabel('satire')
plt.ylabel('propaganda')
plt.show()

### Checking relationships between labels for words

In [ ]:
words = vocab.get_itos()

def collate_word(word):
    text_list, offsets = [], [0]
    processed_text = torch.tensor(text_pipeline(word), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), offsets.to(device)

reliable, hoax, satire, propaganda = [], [], [], []

with torch.no_grad():
    for word in words:
        text, offsets = collate_word(list(word))
        output = model(text, offsets)
        arr = output.numpy()[0]
        reliable.append(arr[3])
        hoax.append(arr[1])
        satire.append(arr[0])
        propaganda.append(arr[2])

In [ ]:
plt.scatter(reliable, hoax)
plt.xlabel('reliable')
plt.ylabel('hoax')
plt.title('Reliable vs hoax - words')
plt.show()

plt.scatter(reliable, satire)
plt.xlabel('reliable')
plt.ylabel('satire')
plt.title('Reliable vs satire - words')
plt.show()

plt.scatter(reliable, propaganda)
plt.title('Reliable vs propaganda - words')
plt.xlabel('reliable')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, propaganda)
plt.title('Hoax vs propaganda - words')
plt.xlabel('hoax')
plt.ylabel('propaganda')
plt.show()

plt.scatter(hoax, satire)
plt.title('Hoax vs satire - words')
plt.xlabel('hoax')
plt.ylabel('satire')
plt.show()

plt.scatter(satire, propaganda)
plt.title('Satire vs propaganda - words')
plt.xlabel('satire')
plt.ylabel('propaganda')
plt.show()

### Get word association graph of words that appear most in each category

In [ ]:
document_frequencies = {}
for key, value in training_dataset.document_frequencies.items():
    df_total = sum(value)
    # Can tweak this value
    if df_total > 500:
        document_frequencies[key] = list(map(lambda x: x / df_total, value))

most_reliable_words = heapq.nlargest(20, document_frequencies, key=lambda x: document_frequencies.get(x)[3])
most_hoax_words = heapq.nlargest(10, document_frequencies, key=lambda x: document_frequencies.get(x)[1])
most_satire_words = heapq.nlargest(30, document_frequencies, key=lambda x: document_frequencies.get(x)[0])
most_propaganda_words = heapq.nlargest(30, document_frequencies, key=lambda x: document_frequencies.get(x)[2])

# 10 hand picked words for each category
reliable_egs = ['taipei', 'taiwanese', 'taiwan', 'yuan', 'premier', 'beijing', 'copyright', 'province', 'ministry', 'chinese']
hoax_egs = ['commented', 'trumps', 'liberals', 'michelle', 'donald', 'trump', 'obamas', 'clintons', 'racist', 'muslims']
satire_egs = ['fucking', 'shit', 'quarterback', 'nfl', 'resident', 'baseball', 'reportedly', 'longtime', 'confirmed', 'noting']
propaganda_egs = ['globalist', 'turbeville', 'codex', 'alimentarius', 'sheeple', 'prepper', 'anti-media', 'attribution', 'anglo-american', 'conspiracies']

### Generate word graph

In [ ]:
concat = np.concatenate([reliable_egs, hoax_egs, satire_egs, propaganda_egs])

x, y = [], []

with torch.no_grad():
    for word in concat:
        text, offsets = collate_word(list(word))
        output = model(text, offsets)
        arr = output.numpy()[0]
        x.append(arr[3] - arr[1])
        y.append(arr[0] - arr[2])

plt.scatter(x, y)
plt.title('Distribution of words')
plt.xlabel('Left - hoax, Right - reliable')
plt.ylabel('Top - propaganda, Bottom - satire')
for idx, label in enumerate(concat):
    plt.annotate(label, (x[idx], y[idx]))
plt.show()